In [54]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import os.path as osp
import numpy as np

In [89]:
def plot_data(data, xaxis='Epoch', value="AverageEpRet", condition="Condition1", smooth=1, **kwargs):
    if smooth > 1:
        """
        smooth data with moving window average.
        that is,
            smoothed_y[t] = average(y[t-k], y[t-k+1], ..., y[t+k-1], y[t+k])
        where the "smooth" param is width of that window (2k+1)
        """
        y = np.ones(smooth)
        for datum in data:
            x = np.asarray(datum[value]["values"])
            z = np.ones(len(x))
            smoothed_x = np.convolve(x,y,'same') / np.convolve(z,y,'same')
            datum[value]["values"] = smoothed_x

    if isinstance(data, list):
        data = pd.concat(data, ignore_index=False, keys=["run_"+ str(i) for i in range(len(data))])

     
    
    sns.set(style="darkgrid", font_scale=1.5)
    # sns.tsplot(data=data, time=xaxis, value=value, unit="Unit", condition=condition, ci='sd', **kwargs)
    sns.lineplot(data=data[value][:], x=xaxis, y="values", ci='sd', **kwargs)

    plt.ylabel(value)
    """
    If you upgrade to any version of Seaborn greater than 0.8.1, switch from 
    tsplot to lineplot replacing L29 with:

        sns.lineplot(data=data, x=xaxis, y=value, hue=condition, ci='sd', **kwargs)

    Changes the colorscheme and the default legend style, though.
    """
    plt.legend(loc='best').set_draggable(True)
    #plt.legend(loc='upper center', ncol=3, handlelength=1,
    #           borderaxespad=0., prop={'size': 13})

    """
    For the version of the legend used in the Spinning Up benchmarking page, 
    swap L38 with:

    plt.legend(loc='upper center', ncol=6, handlelength=1,
               mode="expand", borderaxespad=0., prop={'size': 13})
    """

    xscale = np.max(np.asarray(data[value][xaxis])) > 5e3
    if xscale:
        # Just some formatting niceness: x-axis scale in scientific notation if max x is large
        plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.tight_layout(pad=0.5)

In [90]:
experiment_path = "/home/dwalkerhowell3/star_lab/experiment_copies/experiment_RNN_v_GNN_robotarium_hsn_3_agents_2023-05-30:03-35-23-PM"

sacred_indexs = [1, 2]

environment = "HeterogeneousSensorNetwork-v0"

all_data = []
for sacred_index in sacred_indexs:
    sacred_run_dir = os.path.join(experiment_path, "results", "sacred_runs", environment, str(sacred_index))

    # load the results
    with open(os.path.join(sacred_run_dir, "metrics.json"), 'r') as json_file:
        data = json.load(json_file)
    all_data.append(pd.DataFrame(data))

plot_data(all_data, xaxis="steps", value="test_return_mean")
# plot_data(all_data[1], xaxis="steps", value="test_return_mean")

/tmp/ipykernel_42665/3978941496.py:23: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.lineplot(data=data[value][:], x=xaxis, y="values", ci='sd', **kwargs)


ValueError: Could not interpret value `steps` for parameter `x`